# Reordering a pedigree by *Kahn's* algorithm
 This script is an adaptation of the reord/renum function in __PedWorks.py__ ([more](https://github.com/BrnCPrz/PedWorks)) program. It has the objective of presenting a new method to reorder a pedigree using a network theory framework. The Kahn's algorithm (Kahn, 1962) for network topological sorting performs a linear ordering of its vertices such that for every edge [x,y] connecting vertices x and y, x will come before y in the final ordering. When applied to genealogical structured data (or simply pedigrees!), edge connections [x,y] may be interpreted as the parent-child relationship, where the parent is represented by x and progeny by y. This brings the perception that when applying the algorithm, x (parents) will always come before their progeny (y) in the final ordering.
 
#### References:
Kahn, Arthur B. (1962), "Topological sorting of large networks", Communications of the ACM 5 (11): 558–562, doi:10.1145/368996.369025

In [4]:
import networkx as nx
import numpy as np
from collections import deque
import pandas as pd

In [5]:
# input and visualize pedigree
# missing parentes must be coded "0" (zero)
# animal ids may be numerical or alphanumerical
pedigree = pd.read_table("ped_test.in", header=None, sep= " ")
pedigree.columns = ["id", "sire", "dam"]
print pedigree

   id sire dam
0   A    0   0
1   B    0   0
2   C    0   0
3   D    0   0
4   E    A   B
5   F    A   H
6   G    C   D
7   H    A   D
8   I    G   B
9   J    G   E
10  K    A   H


In [6]:
# define function to get columns names from a file
def getColumns(inFile, delim="\t", header=True):
    '''
        Get columns of data from inFile. The order of the rows is respected

        :param inFile: column file separated by delim
        :param header: if True the first line will be considered a header line
        :returns: a tuple of 2 dicts (cols, indexToName). cols dict has keys that
        are headings in the inFile, and values are a list of all the entries in that
        column. indexToName dict maps column index to names that are used as keys in
        the cols dict. The names are the same as the headings used in inFile. If
        header is False, then column indices (starting from 0) are used for the
        heading names (i.e. the keys in the cols dict)
        '''
    cols = {}
    indexToName = {}
    for lineNum, line in enumerate(inFile):
        if lineNum == 0:
            headings = line.split(delim)
            i = 0
            for heading in headings:
                heading = heading.strip()
                if header:
                    cols[heading] = []
                    indexToName[i] = heading
                else:
                    # in this case the heading is actually just a cell
                    cols[i] = [heading]
                    indexToName[i] = i
                i += 1
        else:
            cells = line.split(delim)
            i = 0
            for cell in cells:
                cell = cell.strip()
                cols[indexToName[i]] += [cell]
                i += 1
    return cols, indexToName

In [7]:
# define a function that receives a pedigree file and translate it into a  network
# uses NetworkX package
def input_diGraph(inFile, animal=0, sire=1, dam=2, rm=True):
    '''
        inFile - pedigree as .txt file.
        animal - column for the animal ID
        sire - column for the sire ID
        dam - column for the dam ID
        rm - True for removing node = "0"

        > "rm" argument eliminates "0" (missing) individuals from graph
        > if not removed, network topological organization will be impaired

        Output:
            pedForGraph: list of tuples - (animal1/animal2, parent)
            PedGraph: networkX graph
    '''
    ped_file = open(inFile, "r")
    cols, indexToName = getColumns(ped_file, delim=' ', header=False)
    ped_file.close()
    # separate animal / sire / dam in lists
    animal_in = cols[animal]
    sire_in = cols[sire]
    dam_in = cols[dam]

    # creates anim_parent(1/2) tuples
    anim_parent1 = zip(sire_in, animal_in)
    anim_parent2 = zip(dam_in, animal_in)

    # create single object containing all parent_child relationship
    pedForGraph = anim_parent1 + anim_parent2

    PedGraph = nx.DiGraph(pedForGraph)
    if rm == True:
        PedGraph.remove_node("0")
        return PedGraph

    else:
        return PedGraph


In [24]:
# A function to apply Kahn's Algorithm to sort a NetworkX (di)graph object
def ped_sort(file):
    """
    - Reorders a pedigree (dict object) using Kahn's Algorithm.
    
    """
    pedgraph = input_diGraph(inFile=file)
    print "\n\tApplying Kahn's Algorithm... "
    in_degree = {u: 0 for u in pedgraph}  # determine in-degree
    for u in pedgraph:  # of each node
        for v in pedgraph[u]:
            in_degree[v] += 1

    Q = deque()  # collect nodes with zero in-degree
    for u in in_degree:
        if in_degree[u] == 0:
            Q.appendleft(u)
    
    order_list = []  # list for order of nodes

    while Q:
        u = Q.pop()  # choose node of zero in-degree
        order_list.append(u)  # and 'remove' it from graph
        for v in pedgraph[u]:
            in_degree[v] -= 1
            if in_degree[v] == 0:
                Q.appendleft(v) 
                
    if len(order_list) == len(pedgraph):
        return order_list
    else:  # if there is a cycle,
        print "Error: At least one cycle detected!\n"
        return []  # return an empty list

In [25]:
ped_sort("ped_test.in")


	Applying Kahn's Algorithm... 
in_degree =  {'A': 0, 'C': 0, 'B': 0, 'E': 2, 'D': 0, 'G': 2, 'F': 2, 'I': 2, 'H': 2, 'K': 2, 'J': 2}
Q before order_list =  deque(['D', 'B', 'C', 'A'])
Q after while =  deque([])
order list =  ['A', 'C', 'B', 'D', 'E', 'H', 'G', 'K', 'F', 'I', 'J']


['A', 'C', 'B', 'D', 'E', 'H', 'G', 'K', 'F', 'I', 'J']